In [ ]:
# !pip install numpy
# !pip install matplotlib
# !pip install pandas
# !pip install seaborn
# !pip install scikit-learn
# !pip install plotly

In [ ]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objs as go
from plotly.subplots import make_subplots

from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso, ElasticNet, SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, r_regression
from sklearn.preprocessing import StandardScaler, RobustScaler

import os

# Stage 1

## 1A : Brief Description of Data

### The World Happiness Report

* The World Happiness Report reviews the state of happiness across a multitude of countries based on a series of questions, with answers rated between 0 and 10. Six variables are then used to explain the overall 'happiness', comparing the outcome against a fictional country called Dystopia, which is a culmination of all the lowest ratings gathered that year.

https://worldhappiness.report/about/

* The data and inforation regrarding it was sourced from : https://worldhappiness.report/data/


In [ ]:
happiness_report = pd.read_excel('data/DataForTable2.1WHR2023.xls')

### 1.1A Summary Statistics: Happiness Report

In [ ]:
happiness_report.describe()

### 1.2A Column Headers: Happiness Report


In [ ]:
list(happiness_report)

### 1.3B Graphs : Happiness Report

In [ ]:
countries = happiness_report['Country name']
happiness = happiness_report['Life Ladder']

plt.figure(figsize=(30, 6))
plt.bar(countries, happiness, color='orange')
plt.xlabel('Country')
plt.ylabel('Life Ladder  (%)')
plt.title('Happiness Across Countries')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## 1B : Brief Description of Data



### World Educational Data

* The World Educational Data contains information on various educational indicators from countries around the world. It includes a range of variables related to education, such as literacy rates, enrollment rates, birth rates, and unemployment rates.

* The data is taken from sources which were last updated in 2021.

* The data and inforation regarding it was sourced from : https://www.kaggle.com/datasets/nelgiriyewithana/world-educational-data


In [ ]:
education_report = pd.read_excel('data/education_world.xls')

### 1.1B Summary Statistics: Education Report

In [ ]:
education_report.describe()

### 1.2B Column Headers : Education Report

In [ ]:
list(education_report)

### 1.3B Graphs : Education Report

In [ ]:
countries = education_report['Countries and areas']
unemployment_rates = education_report['Unemployment_Rate']

plt.figure(figsize=(30, 6))
plt.bar(countries, unemployment_rates, color='orange')
plt.xlabel('Country')
plt.ylabel('Unemployment Rate (%)')
plt.title('Unemployment Rates Across Countries')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


### 1.4 Linking Data from Different Sources

I am going to explore the correlation between educational 'success' and perceived happiness. I also want to examine gender disparity in education and its potential effect on purported happiness.

I aim to develop a predictive model to forecast happiness scores based on gender disparities in education indicators, such as enrollment rates and literacy rates between males and females.

# Stage 2

## Justification for Predicting Happiness Scores

Happiness is an important indicator of human well-being and quality of life. Gaining an understanding of the critical factors that dictate happiness, as well as the correlation between education and happiness, could offer insights into areas of society that could be highlighted and improved

# Stage 3

## 3.1a Clean the Data : Happiness Dataset

### 3.1.1a Check for Missing Values

In [ ]:
print(happiness_report.isnull().sum())

In [ ]:
percent_columns_null = (happiness_report.isnull().sum() / len(happiness_report)) * 100

In [ ]:
print("Percentage of null values in the Data")
print(percent_columns_null.apply(lambda x: "{:.2f}%".format(x)))


The majority of columns have a number of null values, but overall the percentage is low. I decided to perform imputation using the mean value of the column, using SimpleImputer from the scikitlearn module. (scikit-learn, 2024)

In [ ]:
columns_with_missing_values = ['Log GDP per capita', 'Social support', 'Healthy life expectancy at birth', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption', 'Positive affect', 'Negative affect']

imputer = SimpleImputer(strategy='mean')
happiness_report[columns_with_missing_values] = imputer.fit_transform(happiness_report[columns_with_missing_values])

After performing imputation check that the report no longer contains null values.

In [ ]:
happiness_report.isnull().sum()

### 3.1.2a Check for Duplicates

In [ ]:
print(happiness_report.duplicated().sum())

### 3.1.3a Check for Inconsistencies

In [ ]:
print("Number of Unique Country Names: ", happiness_report['Country name'].nunique())
print("Total Number of Country Names: ", happiness_report['Country name'].shape[0])
print("Count of Unique Country Names", happiness_report['Country name'].value_counts())

## 3.1b Clean the Data : Education Dataset

### 3.1.1b Check for Missing Values

In [ ]:
print(education_report.isnull().sum())

### 3.1.1b Check for Duplicates

In [ ]:
print(education_report.duplicated().sum())

### 3.1.3b Check for Inconsistencies

In [ ]:
print("Number of Unique Country Names", education_report['Countries and areas'].nunique())
print("Total Number of Country Names", education_report['Countries and areas'].shape)

### 3.1.4 Collate based on Country Names

hapiness has 165, education has 202. Compare and print unmatching countries

In [ ]:
happiness_countries = happiness_report['Country name'].unique()
education_countries = education_report['Countries and areas'].unique()

matching_countries = set(happiness_countries).intersection(education_countries)

print("Matching Countries:")
for country in matching_countries:
    print(country)

print("Total matching countries: ", len(matching_countries))

## 3.2 Link the Datasets  : Happiness dataset and Education Dataset

* Merge on a common column (country)
* Ensure the column names match
* Remove the log/lat from education as not necessary
* Remove unmatching country rows.

In [ ]:
new_happiness_report = happiness_report[happiness_report['Country name'].isin(matching_countries)]

In [ ]:
new_education_report = education_report[education_report['Countries and areas'].isin(matching_countries)]

In [ ]:
new_education_report['Countries and areas'].nunique()

In [ ]:
new_education_report = new_education_report.drop(columns=['Latitude ', 'Longitude'])

In [ ]:
new_education_report.rename(columns={'Countries and areas': 'Country name'}, inplace=True)
new_education_report.rename(columns={'Countries and areas': 'Country name'}, inplace=True)

Merge the two tables on the column name 'Country name' after changing the column name in the eductaion dataset.

In [ ]:
merged_table = new_happiness_report.merge(new_education_report, on='Country name', how='inner')

As the education report is based off date from 2021 I decided to remove all other years.

In [ ]:
merged_table = merged_table[merged_table['year'] == 2021]
merged_table.drop(columns=['year'], inplace=True)

Graph showing correlation between Happiness score (life ladder) and gross primary vs gross tertiary education using Plotly's Python graphing library. (plotly, 2024) This shows that there is a correlation between third level education and perceived happiness, more significantly affected than primary school enrollment.

In [ ]:
happiness_score = merged_table['Life Ladder']
gross_primary = merged_table['Gross_Primary_Education_Enrollment']
gross_sec = merged_table['Gross_Tertiary_Education_Enrollment']
countries = merged_table['Country name']

fig = make_subplots(rows=1, cols=2, subplot_titles=('Gross_Primary_Education_Enrollment', 'Gross_Tertiary_Education_Enrollment'))

fig.add_trace(go.Scatter(x=happiness_score, y=gross_primary, mode='markers',
                         marker=dict(color='orange'),
                         text=countries,
                         name='Primary'), row=1, col=1)

fig.add_trace(go.Scatter(x=happiness_score, y=gross_sec, mode='markers',
                         marker=dict(color='green'),
                         text=countries,
                         name='Tertiary'), row=1, col=2)

fig.update_layout(title='Happiness Score vs Literacy Rate',
                  xaxis_title='Happiness Score',
                  yaxis_title='Gross Enrollment',
                  legend_title='Level')

fig.show()

# Stage 4 : Choose and train Estimator.

### 4.1 Feature Extraction

#### 4.1.1 Correlation Heatmap
As my dataset currently has more than 33 columns, I decided to generate a heatmap to observe the features with the highest correlation and then to only use these columns in training a model.

In [ ]:
merged = merged_table.iloc[:, 1:]

In [ ]:
corr = merged.corr()
plt.figure(figsize=(30,6))
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            cmap="BuPu",
            vmin=-1,
            vmax=1,
            annot=True)
plt.title("Correlation Heatmap of mtcars dataset")
plt.show()

From the above heatmap we can see there is high correlation between life ladder and the following: Log GPD per capita, Social support, Healthy life expectancy at birth, Lower_Secondary_End_Proficiency_Reading, Gross_Tertiary_Education_Enrollment, and Lower_Secondary_End_Proficiency_Math

In [ ]:
selected_features = ['Log GDP per capita', 'Social support', 'Healthy life expectancy at birth', 'Lower_Secondary_End_Proficiency_Reading', 'Gross_Tertiary_Education_Enrollment', 'Lower_Secondary_End_Proficiency_Math']

#### 4.1.2 SelectKBest
 I also explored feature selection using scikit-learn module, as demonstrated in the tutorial 'How to Build a Predictive Model in Python?' (365 Data Science, 2022). It was necessaryy to remove the Generosity column as it contained negative values.

In [ ]:
selected_features_table = merged_table.drop(columns=['Generosity'])

In [ ]:
X= selected_features_table.iloc[:,2:]
Y= selected_features_table.iloc[:,1]

best_features= SelectKBest(score_func=r_regression, k= 6)
fit= best_features.fit(X,Y)

df_scores= pd.DataFrame(fit.scores_)
df_columns= pd.DataFrame(X.columns)

features_scores= pd.concat([df_columns, df_scores], axis=1)
features_scores.columns= ['Features', 'Score']
features_scores.sort_values(by = 'Score')

The above shows the top six features match the six features highlighted earlier as being the most correlated to the life ladder variable.

In [ ]:
X_features = merged_table[selected_features]

### 4.2 Split the Dataset
Split the updated dataset into training and testing sets

In [ ]:
features = merged_table.columns[1:]
target = merged_table.columns[1]
y = merged_table[target].values

X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

### 4.3 Choose Estimator
The next step was to create the ElasticNet Linear Regression Model. I chose this model based on scikit-learn's machine learning map tutorial (scikit-learn, 2024).
This estimator also highlighted the Lasso Linear Regression option as potentially relevant for my project. I decided to implement and evaluate both to see which would be the better fit.

#### 4.3.1 Create and train the ElasticNet Model

In [ ]:
# params = alpha (weight of penalties) & lambda (overall strength of regularisation)
elastic_model = ElasticNet()
elastic_model.fit(X_train, y_train)

#### 4.3.2 Create and train the Lasso Model
The lasso model also automatically selects relevant features improving generalisation. (Saturn Cloud, 2023)

In [ ]:
lasso_model = Lasso()
lasso_model.fit(X_train, y_train)

### 4.3.3 Create and Train the SGD Model

The Stochastic Gradient Descent Model (SGD)

In [ ]:
sgd_model = SGDRegressor()
sgd_model.fit(X_train, y_train)

#### 4.3.4 Generate Predictions for all Models

In [ ]:
elastic_predictions = elastic_model.predict(X_test)
lasso_predictions = lasso_model.predict(X_test)
sgd_predictions = sgd_model.predict(X_test)

# Stage 5 : Model Evaluation

## 5.1 Evaluate by Calculating Mean Squared Error(MSE)
Calculate MSE to measure the distance between the actual and predicted values for both the Elastic model and the Lasso model. (Daniel, S., (2022))

In [ ]:
elastic_mse = mean_squared_error(y_test, elastic_predictions)
lasso_mse = mean_squared_error(y_test, lasso_predictions)
sgd_mse = mean_squared_error(y_test, sgd_predictions)

In [ ]:
print("Elastic MSE:", elastic_mse)
print("Lasso MSE:", lasso_mse)
print("SGD MSE:", sgd_mse)

The result shows a more accurate result for the Lasso model, then the Elastic model. The SGD model performed the worst with the default parameters.

## 5.2 Evaluate by Calculating the R squared

Calculating the R squared measures the 'goodness of fit' of a regression model by comparing how the model fits the data comapred to a mean line ranging from 0 to 1, where the higher the value the better the fit. (Daniel, S., (2022))

In [ ]:
elastic_r2 = r2_score(y_test, elastic_predictions)
lasso_r2 = r2_score(y_test, lasso_predictions)
sgd_r2 = r2_score(y_test, sgd_predictions)

In [ ]:
print("Elastic R2:", elastic_r2)
print("Lasso R2:", lasso_r2)
print("SGD R2:", sgd_r2)

The result again shows a much better result for the Lasso and Elastic model, while the SGD model performs very poorly.

## 5.3 Fine Tune and Test

### 5.3.1 Fine Tuning the Lasso Model

Created a loop to traverse the number of alpha values to be checked, each time checking the mse and r2, eventually outputting the best possible model and alpha parameter, based off mse and r2 scores.

#### 5.3.1.1 Feature Scaling 

Before fine tuning, I normalised the data using StandardScalar and RobustScalar to determine which would improve the results. Normalisation helps standardise the features, ultimately resulting in better overall model performance.  (atoti, n.d.)

##### 5.3.1.1.1 Robust Scaler

In [ ]:
scaler = RobustScaler()
X_train_robust_scaled = scaler.fit_transform(X_train)
X_test_robust_scaled = scaler.transform(X_test)

##### 5.3.1.1.2 Standard Scaler

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
alpha_values = np.linspace(0.0001, 1, 100)

best_alpha = None
best_lasso_model = None
best_mse = float("inf")

for alpha in alpha_values:
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_train_robust_scaled, y_train)
    lasso_predictions = lasso_model.predict(X_test_robust_scaled)
    lasso_mse = mean_squared_error(y_test, lasso_predictions)
    
    if lasso_mse < best_mse:
        best_alpha = alpha
        best_lasso_model = lasso_model
        best_mse = lasso_mse

lasso_predictions = best_lasso_model.predict(X_test_robust_scaled)
lasso_r2 = r2_score(y_test, lasso_predictions)

print("Best Lasso Model:")
print("Alpha:", best_alpha)
print("MSE:", best_mse)
print("R2:", lasso_r2)


Scaling the data beforehand gave a slight improvement of both MSE and r2 score using the StandardScalar, but the RobustScaler gave a better result.
RobustScaler is normally used in cases of data with outliers. Interestingly, it provided a better outcome in the case of the Lasso model. scikit-learn (n.d.) 


Results Before Normalisation
* Alpha: 0.010199999999999999
* MSE: 0.3856511937788607
* R2: 0.7477302731110707

Standard Scaler Results
* Alpha: 0.0203
* MSE: 0.38404698653221486
* R2: 0.74877964863619

RobustScaler Results
* Alpha: 0.0203
* MSE: 0.37841943260160277
* R2: 0.7524608546483313


### 5.3.2 Fine Tuning the ElasticNet Model

The following works the same as for the Lasso model except that the inner loop additionally iterates over different values of the 'L1 ratio' parameter, 0 - 1. (scikit-learn, n.d.)

#### 5.3.2.1 Feature Scaling

I used the same results from feature scaling earlier for the ElasticNet Model

In [ ]:
alpha_values = np.linspace(0.01, 1, 100)
l1_ratio_values = np.linspace(0, 1, 100)
max_iter = 10000

best_alpha = None
best_l1_ratio = None
best_elastic_model = None
best_mse = float("inf")

for alpha in alpha_values:
    for l1_ratio in l1_ratio_values:
        elastic_model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=max_iter)
        elastic_model.fit(X_train_scaled, y_train)
        elastic_predictions = elastic_model.predict(X_test_scaled)
        elastic_mse = mean_squared_error(y_test, elastic_predictions)

        if elastic_mse < best_mse:
            best_alpha = alpha
            best_l1_ratio = l1_ratio
            best_elastic_model = elastic_model
            best_mse = elastic_mse

best_elastic_model.fit(X_train_scaled, y_train)

elastic_predictions = best_elastic_model.predict(X_test_scaled)
elastic_mse = mean_squared_error(y_test, elastic_predictions)
elastic_r2 = r2_score(y_test, elastic_predictions)

print("Best Elastic Model")
print("Alpha: ", best_alpha)
print("L1 Ratio: ", best_l1_ratio)
print("MSE: ", elastic_mse)
print("R2: ", elastic_r2)


In the case of the ElasticNet Model, scaling the features using StandardScaler not only improved the MSE and R2 results, it also reduced convergance time significantly.

Interestingly, in the case of the ElasticNet model, it also performed better using the RobustScaler. This may be due to a difference in sensitivity to scaling by the model. Ouput as follows:

RobustScaler Results
* Alpha:  0.02
* L1 Ratio:  0.3434343434343435
* MSE:  0.37697763615039054
* R2:  0.7534039908367942

StandardScaler Results
* Alpha:  0.05
* L1 Ratio:  0.21212121212121213
* MSE:  0.3812524170788433
* R2:  0.750607687247626

No Scaler Results
* Alpha:  0.01
* L1 Ratio:  0.9393939393939394
* MSE:  0.3851970694655492
* R2:  0.7480273338186239

### 5.3.3 Fine Tuning the SGD Model

In [ ]:
alpha_values = np.linspace(0.0001, 1, 100)
l1_ratio_values = np.linspace(0, 1, 100)
max_iter = 1000

best_alpha = None
best_sgd_model = None
best_l1_ratio = None
best_mse = float("inf")

random_state = 42
for alpha in alpha_values:
    for l1_ratio in l1_ratio_values:
        sgd_model = SGDRegressor(alpha=alpha, max_iter=max_iter, l1_ratio=l1_ratio, random_state=random_state)
        sgd_model.fit(X_train_robust_scaled, y_train)
        sgd_predictions = sgd_model.predict(X_test_robust_scaled)
        sgd_mse = mean_squared_error(y_test, sgd_predictions)
        
        if sgd_mse < best_mse:
            best_alpha = alpha
            best_sgd_model = sgd_model
            best_l1_ratio = l1_ratio
            best_mse = sgd_mse

sgd_predictions = best_sgd_model.predict(X_test_robust_scaled)
sgd_r2 = r2_score(y_test, sgd_predictions)

print("Best SGD Model:")
print("Alpha:", best_alpha)
print("L1 ratio: ", best_l1_ratio)
print("MSE:", best_mse)
print("R2:", sgd_r2)


Scaling the SGD model features vastly improved the scores. The SGD model is very sensitive to feature scaling and performs much better on normalised data, so this result is not surprising. In the case of the SGD model, the RobustScaler performed slightly better, as can be seen below.

RobustScaler Results
* Alpha: 0.0203
* L1 ratio:  0.0
* MSE: 0.38770354856706724
* R2: 0.7463877465216238

StandardScalar Results
* Alpha: 0.13
* MSE: 0.39239607587197767
* R2: 0.7433181784748374

No Scaling Results
* Best SGD Model:
* Alpha: 1.0
* MSE: 6.542468966072278e+23
* R2: -4.279688189418018e+23

### 5.3.4 Scatter Plot of Actual vs Predicted Values of All Models

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 6))

axes[0].scatter(y_test, lasso_predictions, color='blue', alpha=0.6)
axes[0].plot(y_test, y_test, color='red', linewidth=0.5)
axes[0].set_title('Lasso Model')
axes[0].set_xlabel('Actual Values')
axes[0].set_ylabel('Predicted Values')
axes[0].grid(True)

axes[1].scatter(y_test, elastic_predictions, color='green', alpha=0.6)
axes[1].plot(y_test, y_test, color='red', linewidth=0.5)
axes[1].set_title('Elastic Model')
axes[1].set_xlabel('Actual Values')
axes[1].set_ylabel('Predicted Values')
axes[1].grid(True)

axes[2].scatter(y_test, sgd_predictions, color='hotpink', alpha=0.6)
axes[2].plot(y_test, y_test, color='red', linewidth=0.5)
axes[2].set_title('SGD Model')
axes[2].set_xlabel('Actual Values')
axes[2].set_ylabel('Predicted Values')
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Stage 6 : Results and Conclusions

# References
* scikit-learn. (2024). Choosing the right estimator [Online] Available at: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html [Accessed 11 April 2024].

* Sarah El Shatby. (2022). Predictive Model in Python. [Online] Available at: https://365datascience.com/tutorials/python-tutorials/predictive-model-python/ [Accessed 11 April 2024].

* Saturn Cloud. (2023). Python Classification with Lasso: How to Predict Classes. [Online] Available at: https://saturncloud.io/blog/python-classification-with-lasso-how-to-predict-classes/#:~:text=Feature%20Selection%3A%20Lasso%20automatically%20selects,feature%2Dto%2Dsample%20ratio. [Accessed 11 April 2024].

* scikit-learn developers. (2024). Imputation of missing values. [Online] Available at: https://scikit-learn.org/stable/modules/impute.html [Accessed 11 April 2024]

* plotly (2024). Scatter Plots in Python [Online] Available at: https://plotly.com/python/line-and-scatter/ [Accessed 13 April 2024].

* scikit-learn (n.d.) 'Preprocessing data', scikit-learn Documentation, [Online]. Available at: https://scikit-learn.org/stable/modules/preprocessing.html (Accessed: 13 April 2024).

* IBM Developer. (2024). Lasso regression: Automatic feature selection. [Online] Available at: https://developer.ibm.com/tutorials/awb-lasso-regression-automatic-feature-selection/ [Accessed 12 April 2024].

* Stephy Daniel. (2022). Model Evaluation for Classification and Regression Analysis. [Online] Available at: https://medium.com/@stephy.SD99/model-evaluation-for-classification-and-regression-analysis-6fa2d11a4a09 [Accessed 12 April 2024].

* LinkedIn. (n.d.) 'How Do You Interpret Coefficients - Elastic?', LinkedIn. Available at: https://www.linkedin.com/advice/0/how-do-you-interpret-coefficients-elastic#:~:text=The%20parameters%20of%20elastic%20net%20regression%20are%20alpha%20and%20lambda,the%20performance%20of%20the%20model. (Accessed: 17 April 2024).

* Hackernoon. (2018) 'An Introduction to Ridge, Lasso, and Elastic Net Regression,' Hacker Noon. Available at: https://hackernoon.com/an-introduction-to-ridge-lasso-and-elastic-net-regression-cca60b4b934f (Accessed: 17 April 2024).

* Brownlee, J. (2023) 'k-Fold Cross-Validation', Machine Learning Mastery, [Online]. Available at: https://machinelearningmastery.com/k-fold-cross-validation/ (Accessed: 17 April 2024).

* scikit-learn (n.d.) 'ElasticNet', scikit-learn Documentation, [Online]. Available at: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html (Accessed 11 April 2024).

* atoti (n.d.) 'When to Perform a Feature Scaling', atoti Articles, [Online]. Available at: https://www.atoti.io/articles/when-to-perform-a-feature-scaling/#:~:text=In%20regression%2C%20it%20is%20often,are%20set%20to%20their%20means (Accessed 11 April 2024).

* ai-jobs.net (2023) 'ElasticNet Explained', AI Jobs, [Online]. Available at: https://ai-jobs.net/insights/elasticnet-explained/#:~:text=Data%20Scaling%3A%20It%20is%20essential,from%20dominating%20the%20regularization%20process (Accessed: 21 April 2024).

In conclusion, all the models yielded almost identical results after fine-tuning, with the best Lasso model having an alpha value of 0.0203 and achieving an MSE of 0.3784 and an R2 of 0.7525. The best ElasticNet model had an alpha of 0.05 and an L1 ratio of 0.2121, resulting in an MSE of 0.3813 and an R2 of 0.7506. Similarly, the SGD model with an alpha of 0.0203 and an L1 ratio of 0.0 achieved an MSE of 0.3877 and an R2 of 0.7464.

During the fine-tuning stage, testing a range of values from 0.0001 to 1 for the SGD and Lasso models, and 0.01 to 1 for the ElasticNet model, found that the optimal alpha values were 0.0203 fro both, leading to a slight improvement in predictions. Additionally, the best alpha ratio for the ElasticNet model was found to be 0.5.

The MSE results for both models after fine-tuning showed a significant improvement compared to the initial values of 0.77, with the refined models achieving approximately 0.39. This suggests that the predicted life ladder values are much closer to the actual values, as a lower MSE indicates better model performance.

Furthermore, the R-squared (R2) values indicated that all models can predict approximately 75% of the variance in the target variable. This represents a notable improvement from the pre-fine-tuning R2 values of 0.49 for both the Lasso and Elastic, and -4.279688189418018e+23 for the SGD.

The biggest improvement through feature scaling and fine tuning was for the SGD model, leading to near identcal results as teh other two models.

Despite the relatively close approximation, there remains a discernible gap between the predicted and actual values. Further investigation into the factors contributing to this variance is necessary to enhance the accuracy of the models and improve their predictive capabilities.